### Load libraries
Make sure that you have the latest version of *phase_portraits.py* in the same directory as you're running this Jupyter notebook from. It can be downloaded from: 
[https://mitt.uib.no/courses/45546/files/folder/Jupyter%20files](https://mitt.uib.no/courses/45546/files/folder/Jupyter%20files) 

In [6]:
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
import phase_portraits as pp
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import pandas as pd
from IPython.display import Image
#import seaborn as sns


%matplotlib inline

We are going to look at the following SIR modell with vaccination:

![title](Figs/SIR.png)

This is an example of a compartment model, simulating how people are moving from one compartment to the other. When susceptible, $S$, meet infected (\textit{I}) they have a certain probability of being infected. After a while the infected recover, entering the \textit{R} compartment. In this model the recovered are again transferred back to the \textit{S} compartment, they are only immune for a certain amount of time. In the model we have also an arrow bringing individuals from \textit{S} to \textit{R} directly, assuming that the vaccination is only efficient for the same time as for the individual recovered after an infection. The vaccination effort, denoted $V$ in the figure. We're later going to assume that $V$ is either a constant, i.e. a continuous vaccination program, or proportional to the infection level, i.e. reaction to an outbreak.
$$
\begin{array}{rl}
\frac{dS}{dt}  & = - \frac{\beta}{N} I S + \mu R - U\nonumber\\
\frac{dI}{dt}  & =  \frac{\beta}{N} I S- \gamma I  \\
\frac{dR}{dt}  & = \gamma I -\mu R + U \nonumber
\end{array}
$$
where $S$, $I$ and $R$ are, respectively, the fraction of the population that are susceptible, infected, and recovered. This is an example of a compartment model, simulating how people are moving from one compartment to the other. When susceptible, $S$, meet infected *I* they have a certain probability of being infected. *V* represent vaccination, i.e. moving individuals directly from *S* to *R*. 








In [7]:
# Define the function.
def rhs_SI(t,xy,gamma = 0.25, beta = 0.5,mu=0.05,U=0):
    S,I=xy
    #phi,mu,U=args
    dx= - beta*I*S + mu*(1. - S - I)- U
    dy= beta*I*S - gamma*I
    return [dx,dy]

def stop_condition(t, xy,phi=0,mu=0.5,U=0):
    return [1-xy[0]-xy[1]]

In [8]:
#Denne er ikke i bruk, bruker run_ensemble_pdf
def run_ensemble(l_num=1000, gamma = 0.25, beta_mean = .5, beta_std=0, mu = 0.05, U=0,t_end=40., y0=[0.99,0.01],t_num=41,dist=0):
# A function that runs the ensembles for the pendulum
# Input: l_num, the number of samples in the ensemble
#       length_mean, the mean length of the pendulum
#       length_std, standard deviation of the length of the pendulum
#       g, is the gravitational constant
#       dist, the distribution function to use
# Output: t time when we do the statistics. 
#         x_mean, the mean values
#         Standard deviations.             
    
#    beta=np.random.normal(loc=beta_mean,scale=beta_std, size=l_num)
    beta=np.random.uniform(low=np.maximum(0.,beta_mean-beta_std),high=beta_mean+beta_std, size=l_num)
#Create the time vector, this should be an input. 
            



#Create the time vector, this should be an input. 
    
    t = np.linspace(0, t_end, t_num)



# Store the results. 
    S=t*0
    S2=t*0
    I=S
    I2=S2
    R=S
    R2=S2


    
# Here we loop through the samples. 
    stop_condition.terminal = True
    for ii in np.arange(len(mu)):
        tmp=sc.integrate.solve_ivp(rhs_SI,t_span=(0.,t_end),y0=y0,
                                   t_eval=t,args=[gamma,beta[ii],mu,U])
#                                   events=stop_condition)
        #Sum the x and the x squared. 
        S=S+tmp.y[0]
        I=I+tmp.y[1]
        R=R+(1-tmp.y[0]-tmp.y[1])
        S2=S2+tmp.y[0]**2
        I2=I2+tmp.y[1]**2
        R2=R2+(1-tmp.y[0]-tmp.y[1])**2
# and do the statistics
    S=S/l_num
    S2=np.sqrt((S2/l_num)-S**2)
    I=I/l_num
    I2=np.sqrt((I2/l_num)-I**2)
    R=I/l_num
    R2=np.sqrt((R2/l_num)-R**2)

    df = pd.DataFrame({
    'S': S,
    'I': I,
    'R': R,
    'S2': S2,
    'I2': I2,
    'R2': R2,
    'day': t})


    return df
    




    



In [9]:

def run_ensemble_pd(l_num=1000, gamma = 0.25, beta_mean = .5, beta_std=0, mu = 0.05, U=0,t_end=40., y0=[0.99,0.01],t_num=41,dist=0):
# A function that runs the ensembles for the pendulum
# Input: l_num, the number of samples in the ensemble
#       length_mean, the mean length of the pendulum
#       length_std, standard deviation of the length of the pendulum
#       g, is the gravitational constant
#       dist, the distribution function to use
# Output: t time when we do the statistics. 
#         x_mean, the mean values
#         Standard deviations.             
    np.random.seed(0)
    
#    beta=np.random.normal(loc=beta_mean,scale=beta_std, size=l_num)
    beta=np.random.uniform(low=np.maximum(0.,beta_mean-beta_std),high=beta_mean+beta_std, size=l_num)
#Create the time vector, this should be an input. 
    
    t = np.linspace(0, t_end, t_num)



# Store the results. 
    df_list=[]
    
# Here we loop through the samples. 

    stop_condition.terminal = True
    for ii in np.arange(len(beta)):
        tmp=sc.integrate.solve_ivp(rhs_SI,t_span=(0.,t_end),y0=y0,
                                   t_eval=t,args=[gamma,beta[ii],mu,U])
#                                   , events=stop_condition)
        #Sum the x and the x squared. 
        df_loc = pd.DataFrame({
        'S': tmp.y[0],
        'I': tmp.y[1],
        'R': 1-tmp.y[0]-tmp.y[1],
        'day': t})
        df_loc.index = t
        df_list.append(df_loc)
    df= pd.concat(df_list,ignore_index=True)

    return df
    




    



In [11]:
def plot_func(gamma = 0.25, beta_mean = .5, beta_std=0, mu = 0.05, U=0):
    
    y0=[0.99,0.01]
    l_num=1000,
    t_end=40.
    t_num=41
    dist=0
    

    res=run_ensemble_pd(t_end=t_end,t_num=t_num,gamma=gamma,beta_mean = beta_mean, beta_std = beta_std, 
              mu = mu, U=U)

    fig, ax = plt.subplots(1, 1)
    ax.set_xlabel('S')
    ax.set_ylabel('I')
   
    
    ax=sns.lineplot(data=res,x='day',y='S',ax=ax,label='S')
    ax=sns.lineplot(data=res,x='day',y='I',ax=ax,label='I')
    ax=sns.lineplot(data=res,x='day',y='R',ax=ax,label='R')
#    ax.legend(['S','I','R'])
    return ax

gamma_slider=widgets.FloatSlider(value=0.05,
                                               min=0,
                                               max=.5,
                                               step=0.1)
mu_slider=widgets.FloatSlider(value=0.,
                                               min=0,
                                               max=1.0,
                                               step=0.1)

beta_slider=widgets.FloatSlider(value=0.5,
                                               min=0,
                                               max=1.9,
                                               step=0.1)

betastd_slider=widgets.FloatSlider(value=0,
                                               min=0,
                                               max=2.9,
                                               step=0.05)

U_slider=widgets.FloatSlider(value=0,
                                               min=0,
                                               max=0.1,
                                               step=0.01)

interact(plot_func, mu = mu_slider,
         beta_mean=beta_slider,beta_std=betastd_slider,gamma=gamma_slider,
         U=U_slider)

interactive(children=(FloatSlider(value=0.05, description='gamma', max=0.5), FloatSlider(value=0.5, descriptio…

<function __main__.plot_func(gamma=0.25, beta_mean=0.5, beta_std=0, mu=0.05, U=0)>